In [ ]:
import pydgraph
import json

def create_client_stub():
    """Creates a client stub."""
    return pydgraph.DgraphClientStub('localhost:9090')

def create_client(client_stub):
    """Creates a Dgraph client."""
    return pydgraph.DgraphClient(client_stub)

def set_schema(client):
    """Sets the schema in Dgraph."""
    schema = """
        name: string @index(term) .
        age: int .
        skills: [uid] @reverse .
        skill.name: string @index(term) .
        skill.experience: int .
    """
    operation = pydgraph.Operation(schema=schema)
    client.alter(operation)

def import_json_data(client, json_data):
    """Imports JSON data into Dgraph."""
    mutation = pydgraph.Mutation(set_json=json.dumps(json_data).encode('utf-8'))
    request = pydgraph.Request()
    request.mutations.extend([mutation])
    txn = client.txn(read_only=False)
    try:
        assigned = txn.mutate(request=request)
        txn.commit()
        print("JSON data imported successfully!")
        return assigned
    except Exception as e:
        txn.discard()
        print(f"Error importing JSON data: {e}")
        return None

def query_dgraph(client):
    """Queries Dgraph to verify the imported data."""
    query = """
    query allPeople {
      allPeople(func: has(name)) {
        uid
        name
        age
        skills {
          name
          experience
        }
      }
    }
    """
    txn = client.txn(read_only=True)
    try:
        res = client.query(request=pydgraph.Request(query=query))
        txn.commit()
        query_result = json.loads(res.json)
        print("Query Results:")
        print(json.dumps(query_result, indent=2))
        return query_result
    except Exception as e:
        txn.discard()
        print(f"Error during query: {e}")
        return None

if __name__ == "__main__":
    client_stub = create_client_stub()
    client = create_client(client_stub)

    set_schema(client)

    json_str = """
    [
      {
        "uid": "_:person1",
        "name": "Alice",
        "age": 30,
        "skills": [
          {
            "name": "Python",
            "experience": 5
          },
          {
            "name": "Dgraph",
            "experience": 2
          }
        ]
      },
      {
        "uid": "_:person2",
        "name": "Bob",
        "age": 25,
        "skills": [
          {
            "name": "Java",
            "experience": 3
          }
        ]
      }
    ]
    """
    json_data = json.loads(json_str)

    assigned_uids = import_json_data(client, json_data)
    if assigned_uids:
        print("Assigned UIDs:", assigned_uids.uids)

    query_dgraph(client)

    client_stub.close()
    print("Client stub closed.")